In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta

In [2]:
pos_tokens = m.BalancegameRecord.objects.filter(who=660)[:].values_list('pix_0__tokens', flat=True); pos_tokens
neg_tokens = m.BalancegameRecord.objects.filter(who=660)[:].values_list('pix_1__tokens', flat=True); neg_tokens;

In [3]:
pos_stat = collections.Counter(' '.join(pos_tokens).split(' ')); pos_stat
neg_stat = collections.Counter(' '.join(neg_tokens).split(' ')); neg_stat;

In [5]:
(pos_stat - neg_stat).most_common(10)

[('denim', 28),
 ('청', 28),
 ('격식있는옷차림', 28),
 ('fashion_design', 26),
 ('패션디자인', 26),
 ('neck', 23),
 ('목', 23),
 ('pocket', 23),
 ('포켓', 23),
 ('블랙', 23)]

In [37]:
tokens = m.BalancegameRecord.objects.filter(who__user__email='magrittelim@gmail.com').annotate(
    pos_tokens=Case(
        When(chosen=0, then=F('pix_0__tokens')),
        When(chosen=1, then=F('pix_1__tokens')),
        default=Value(''),
    ),
    neg_tokens=Case(
        When(chosen=0, then=F('pix_1__tokens')),
        When(chosen=1, then=F('pix_0__tokens')),
        default=Value(''),
    ),
)

In [38]:
pos_tokens = tokens.values_list('pos_tokens', flat=True)
neg_tokens = tokens.values_list('neg_tokens', flat=True)

In [39]:
pos_stat = collections.Counter(' '.join(pos_tokens).split(' ')); pos_stat
neg_stat = collections.Counter(' '.join(neg_tokens).split(' ')); neg_stat;

In [41]:
(pos_stat - neg_stat).most_common(20)

[('티셔츠', 41),
 ('스니커즈', 25),
 ('t-shirt', 21),
 ('캡', 21),
 ('맨투맨', 21),
 ('cap', 19),
 ('운동화', 18),
 ('sportswear', 15),
 ('운동복', 15),
 ('event', 14),
 ('행사', 14),
 ('shoulder', 13),
 ('어깨', 13),
 ('fun', 13),
 ('장난', 13),
 ('thigh', 13),
 ('대퇴골', 13),
 ('선반', 12),
 ('후드', 11),
 ('human_leg', 11)]